# Data Type Conversion

In [1]:
## Packages
import swat
import sys
import os
import pandas as pd
import numpy as np
import json


## Options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

## My custom connection package for CAS
try:
    from casauth import CASAuth
    print('Imported personal custom CAS auth package')
except:
    print('casauth package not available')

    
## View versions of packages
print(f'Python version:{sys.version.split("|")[0]}')
print(f'swat version:{swat.__version__}')
print(f'pandas version:{pd.__version__}')
print(f'numpy version:{np.__version__}')

Imported personal custom CAS auth package
Python version:3.11.5 
swat version:1.13.3
pandas version:2.1.4
numpy version:1.26.4


In [2]:
###################################################
## My Personal connection using a custom package ##
###################################################
path = os.getenv('CAS_CREDENTIALS')
pem_file = os.getenv('CAS_CLIENT_SSL_CA_LIST')

conn = CASAuth(path, ssl_ca_list = pem_file)

CAS Connection created


In [ ]:
## Create sample DataFrame
df = pd.DataFrame({'col1':['123,345','456,123','789'], 'col2':['$100','$200','$300']})

## Load DataFrame as a distributed CAS table
castbl = conn.upload_frame(df, casout={'name':'testtbl','caslib':'casuser','replace':True})

In [34]:
conn.tableInfo(caslib='casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime,TableRedistUpPolicy
0,COL_CONVERT,3,4,0,utf-8,2024-04-23T15:57:21+00:00,2024-04-23T15:57:21+00:00,2024-04-23T15:57:21+00:00,UTF8,2.029507e+09,2.029507e+09,2.029507e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN,Not Specified
1,TESTTBL,3,2,0,utf-8,2024-04-23T15:58:34+00:00,2024-04-23T15:58:34+00:00,2024-04-23T15:58:34+00:00,UTF8,2.029507e+09,2.029507e+09,2.029507e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2024-04-23T15:58:34+00:00,2.029507e+09,Not Specified


In [35]:
type(castbl)

swat.cas.table.CASTable

In [36]:
castbl.head()

,col1,col2
0,"123,345",$100
1,"456,123",$200
2,789,$300


In [37]:
castbl.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,col1,,1,varchar,7,7,,0,0
1,col2,,2,varchar,4,4,,0,0


Use the INPUT function to convert a character to numeric

[INPUT function documentation](https://go.documentation.sas.com/doc/en/pgmsascdc/v_050/lefunctionsref/p19en16vskd2vhn1vwmxpxnglxxs.htm)

Use the SWAT eval method to create a new column

[SWAT EVAL METHOD Documentation](https://sassoftware.github.io/python-swat/generated/swat.cas.table.CASTable.eval.html#swat.cas.table.CASTable.eval)

In [ ]:
## Eval method adds attributes to the object to create the new column

(castbl
 .eval('col1_num = input(col1,comma16.)', inplace=False)         ## Inplace=False returns another CASTable object
 .eval('col2_num = input(col2, dollar16.)', inplace=False)       ## Inplace=False returns another CASTable object
 .params
)

In [39]:
## Add the head method to preview and make sure it works

(castbl
 .eval('col1_num = input(col1,comma16.)', inplace=False)         ## Inplace=False returns another CASTable object
 .eval('col2_num = input(col2, dollar16.)', inplace=False)       ## Inplace=False returns another CASTable object
 .head()
)

,col1,col2,col1_num,col2_num
0,"123,345",$100,123345.0,100.0
1,"456,123",$200,456123.0,200.0
2,789,$300,789.0,300.0


In [40]:
## Add the columninfo action to view data types

(castbl
 .eval('col1_num = input(col1,comma16.)', inplace=False)         ## Inplace=False returns another CASTable object
 .eval('col2_num = input(col2, dollar16.)', inplace=False)       ## Inplace=False returns another CASTable object
 .columnInfo()
)

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,col1,,1,varchar,7,7,,0,0
1,col2,,2,varchar,4,4,,0,0
2,col1_num,,3,double,8,12,,0,0
3,col2_num,,4,double,8,12,,0,0


In [ ]:
## Add the copyTable actiont o create a new CAS table with updated columns

(castbl
 .eval('col1_num = input(col1,comma16.)', inplace=False)         ## Inplace=False returns another CASTable object
 .eval('col2_num = input(col2, dollar16.)', inplace=False)       ## Inplace=False returns another CASTable object
 .copyTable(casout={'name':'col_convert','caslib':'casuser','replace':True})
)

In [43]:
conn.tableInfo(caslib='casuser')

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,ModTime,AccessTime,Global,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime,TableRedistUpPolicy
0,TESTTBL,3,2,0,utf-8,2024-04-23T15:58:34+00:00,2024-04-23T15:58:34+00:00,2024-04-23T15:58:50+00:00,UTF8,2.029507e+09,2.029507e+09,2.029507e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,2024-04-23T15:58:34+00:00,2.029507e+09,Not Specified
1,COL_CONVERT,3,4,0,utf-8,2024-04-23T15:58:50+00:00,2024-04-23T15:58:50+00:00,2024-04-23T15:58:50+00:00,UTF8,2.029507e+09,2.029507e+09,2.029507e+09,0,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN,Not Specified
